In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import torch
import numpy as np
import pandas as pd
import random

In [ ]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("PyTorch version:", torch.__version__)

CUDA available: True
CUDA version: 12.1
PyTorch version: 2.4.0+cu121


In [ ]:
# data의 root-path
root_path = '/content/drive/MyDrive/BPR/'

## 데이터 생성

In [ ]:
df = pd.read_csv(root_path + "philadelphia_rating_BPR.csv")
df.head()

,newUserId,newBusinessId
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


### split_traint_test

In [ ]:
# 각 사용자별로 n번째 음식점 방문기록 까지 데이터를 테스트 데이터로, 나머지를 트레이닝 데이터로 분리하는 함수
def split_train_test_data(df, n=3, user_col='newUserId', item_col='newBusinessId', output_dir='.'):
    train_data = []
    test_data = []

    # 사용자별로 그룹화하여 처리
    for user_id, group in df.groupby(user_col):
        # 첫 번째부터 n번째 음식점 방문 기록을 test_data로 추가
        test_data.extend(group.iloc[:n].values.tolist())

        # n+1번째부터 마지막 음식점 방문 기록을 train_data로 추가
        train_data.extend(group.iloc[n:].values.tolist())

    train_df = pd.DataFrame(train_data, columns=[user_col, item_col])
    test_df = pd.DataFrame(test_data, columns=[user_col, item_col])


    train_size = train_df.shape
    test_size = test_df.shape
    print("Train 데이터 크기:", train_size)
    print("Test 데이터 크기:", test_size)
    print("Test 데이터 크기 / Train 데이터 크기 비율:", test_size[0] / train_size[0]) # 전체 크기의 대략 10%

    # CSV 파일로 저장
    train_df.to_csv(output_dir + "train.csv", index=False, header=False, sep='\t')
    test_df.to_csv(output_dir + "test.csv", index=False, header=False, sep='\t')

    print("'train.csv'와 'test.csv' 파일이 생성되었습니다.")

split_train_test_data(df, n=3, output_dir=root_path)

Train 데이터 크기: (296794, 2)
Test 데이터 크기: (36185, 2)
Test 데이터 크기 / Train 데이터 크기 비율: 0.12191958058451316
'train.csv'와 'test.csv' 파일이 생성되었습니다.


### generate_negative

In [ ]:
# 각 사용자별로 부정적인 조합 생성
def generate_negative_samples(df, n=100, user_col='newUserId', item_col='newBusinessId', output_dir='.'):
    # 사용자와 음식점 집합 구하기
    users = df[user_col].unique()
    businesses = df[item_col].unique()

    # 사용자별로 부정적인 조합 저장
    user_negative_samples = {}

    # 각 사용자에 대해 부정적인 조합 생성
    for user in users:
        # 사용자가 방문한 음식점 집합
        user_visited_businesses = set(df[df[user_col] == user][item_col].unique())

        # 사용자가 방문하지 않은 음식점 찾기
        user_not_visited_businesses = list(set(businesses) - user_visited_businesses)

        # 랜덤 샘플링하여 부정적인 조합 생성
        if len(user_not_visited_businesses) > n: #n개 이상이면 데이터가 많아지니 제한을 둠
            user_negative_samples[user] = random.sample(user_not_visited_businesses, n)
        else:
            user_negative_samples[user] = user_not_visited_businesses

    # 부정적인 조합을 CSV 파일로 저장
    with open(output_dir+ "negative.csv", 'w') as f:
        for user, negatives in user_negative_samples.items():
            f.write(f"{user},{','.join(map(str, negatives))}\n")

    print("파일 'negative.csv'가 생성되었습니다.")

generate_negative_samples(df, n=100, output_dir=root_path)

파일 'negative.csv'가 생성되었습니다.


### generate_individual_negative

In [ ]:
# 개인별로 방문하지 않은 음식점 샘플링
def generate_individual_negative_sample(df, user_id, n=100, user_col='newUserId', item_col='newBusinessId'):
    # 모든 음식점 목록 가져오기
    businesses = df[item_col].unique()
    # 사용자가 방문한 음식점 목록 가져오기
    user_visited_businesses = set(df[df[user_col] == user_id][item_col].unique())
    # 사용자가 방문하지 않은 음식점 목록 계산
    user_not_visited_businesses = list(set(businesses) - user_visited_businesses)
    # 방문하지 않은 음식점 중에서 n개를 랜덤으로 샘플링
    if len(user_not_visited_businesses) > n:
        random_negative = random.sample(user_not_visited_businesses, n)
    else:
        random_negative = user_not_visited_businesses
    return random_negative


# testData를 위한 부정적인 음식점 샘플링
with open(root_path + 'negative.csv', 'r') as negative_file \
  , open(root_path + 'test.csv', 'r') as test_file \
  , open(root_path + 'test_negative.csv', 'w') as test_negative_file:
    read_negative = negative_file.readlines()
    read_test = test_file.readlines()
    print(len(read_negative))  # 12062 -> 사용자 수만큼 개인별로 방문하지 않은 음식점 집합 완성

    for i in range(len(read_negative)):  # 각 사용자에 대해
        negative = read_negative[i]  # 사용자 i에 대한 방문하지 않은 음식점 집합 가져오기
        user_id, user_id_negative_businesses = negative.strip().split(',', 1)  # 사용자 ID와 방문하지 않은 음식점 구분
        #user_id_negative_businesses = user_id_negative_businesses.split(',')  # 방문하지 않은 음식점들을 리스트로 분리

        for j in range(i * 3, (i + 1) * 3):  # 각 사용자의 3개의 음식점 방문기록
            test = read_test[j % len(read_test)]  # 테스트 데이터 중 해당 사용자의 음식점 방문기록
            user_trip = test.strip()  # 공백 제거

            # 개별 사용자의 부정적인 음식점 생성
            individual_negative_sample = generate_individual_negative_sample(df, int(user_id), n=100)

            # 테스트 데이터와 부정적인 샘플을 결합하여 git에 있는 BPR모델 입력에 맞게 변환
            test_negative_business = user_trip + '\t' + '\t'.join(map(str, individual_negative_sample)) + '\n'
            test_negative_file.write(test_negative_business)

print("파일 'test_negative.csv'가 생성되었습니다.")

12062
파일 'test_negative.csv'가 생성되었습니다.


## 모델 생성

### 평가함수

In [ ]:
def hit(gt_item, pred_items):
	if gt_item in pred_items:
		return 1
	return 0


def ndcg(gt_item, pred_items):
	if gt_item in pred_items:
		index = pred_items.index(gt_item)
		return np.reciprocal(np.log2(index+2))
	return 0


def metrics(model, test_loader, top_k):
	# HR: 전체 사용자 수 대비 적중한 사용자 수
	# NDCG: '사용자'와 '추천한 아이템'의 '관련성'에 대해 추천 순서에 따라 가중치를 주어 합한뒤 정규화한 것
	HR, NDCG = [], []

	for user, item_i, item_j in test_loader:
		user = user.cuda()
		item_i = item_i.cuda()
		item_j = item_j.cuda() # not useful when testing

		prediction_i, prediction_j = model(user, item_i, item_j)
		_, indices = torch.topk(prediction_i, top_k)
		recommends = torch.take(
				item_i, indices).cpu().numpy().tolist()

		gt_item = item_i[0].item()
		HR.append(hit(gt_item, recommends))
		NDCG.append(ndcg(gt_item, recommends))

	return np.mean(HR), np.mean(NDCG)

### 모델

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import os
import time
import numpy as np
import torch.backends.cudnn as cudnn

import pandas as pd
import scipy.sparse as sp
import random
import torch.utils.data as data

In [ ]:
# 하드코딩된 기본값 설정
lr = 0.001
lamda = 0.001
batch_size = 4096
epochs = 10
top_k = 10
num_factors = 32
num_ng = 4
test_num_ng = 99
out = True

In [ ]:
class BPRData(data.Dataset):
    def __init__(self, features, num_item, train_mat=None, num_ng=0, is_training=None):
        super(BPRData, self).__init__()
        self.features = features # train_data
        self.num_item = num_item # 아이템 수
        self.train_mat = train_mat # 훈련 데이터의 희소 행렬
        self.num_ng = num_ng # 부정적 샘플의 수
        self.is_training = is_training # 훈련모드의 여부

    def ng_sample(self): # 부정적 샘플링
        assert self.is_training, 'no need to sampling when testing'

        self.features_fill = []
        for x in self.features:
            u, i = x[0], x[1]
            for t in range(self.num_ng):
                j = np.random.randint(self.num_item)
                while (u, j) in self.train_mat: #train_mat를 사용하여 (u,j)라는 특정 사용자-아이템 쌍이 train_data에 존재하는지 여부를 빠르게 확인함.
                    j = np.random.randint(self.num_item) # (u,j)가 희소행렬에 존재 한다면 방문하지 않은 음식점 j를 찾을 때까지 랜덤값으로 값 획득
                self.features_fill.append([u, i, j]) #triple_data를 획득

    def __len__(self):
        if self.is_training:
            return self.num_ng * len(self.features)
        else:
            return len(self.features)

    def __getitem__(self, idx):
        features = self.features_fill if self.is_training else self.features

        user = features[idx][0]
        item_i = features[idx][1]
        item_j = features[idx][2] if self.is_training else features[idx][1]
        return user, item_i, item_j


class BPR(nn.Module):
    def __init__(self, num_users, num_items, num_factors=10):
        super(BPR, self).__init__()
        self.user_factors = nn.Embedding(num_users, num_factors) # 초기 임베딩값을 무작위로 넣는게 사전확률로 볼 수 있다.
        self.item_factors = nn.Embedding(num_items, num_factors)

    def forward(self, user_ids, item_ids_i, item_ids_j):
      user_embedding = self.user_factors(user_ids) # # 파라미터 p(u)
      item_embedding_i = self.item_factors(item_ids_i) #파라미터 q(i)
      item_embedding_j = self.item_factors(item_ids_j)# 파라미터 q(j)

      # 내적을 통한 예측값 계산

      #x(uij)를 구하기 위한과정
      pred_i = torch.sum(user_embedding * item_embedding_i, dim=-1)  #아이템 i에 대한 점수 x(ui) -> p(u)*(q(i)^T)
      pred_j = torch.sum(user_embedding * item_embedding_j, dim=-1) #아이템 j에 대한 점수 x(uj)  -> p(u)*(q(j)^T)

      return pred_i, pred_j

### 데이터셋 준비

In [ ]:
def load_all():
    # 학습 데이터 로드
    train_data = pd.read_csv(
        root_path + 'train.csv',
        sep='\t', header=None, names=['user', 'item'],
        usecols=[0, 1], dtype={0: np.int32, 1: np.int32})


    # 사용자 및 아이템 수 계산
    user_num = train_data['user'].max() + 1
    item_num = train_data['item'].max() + 1

    # 학습 데이터를 리스트로 변환
    train_data = train_data.values.tolist()

    # 학습 데이터를 scipy sparse matrix로 변환
    train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32) # 희소 행렬은 메모리 사용량을 줄이고, 특정 요소에 대한 빠른 조회를 가능하게 함.
    for x in train_data: #후속 단계에서 부정적 샘플링을 수행할 때 유용
        train_mat[x[0], x[1]] = 1.0 # 사용자와 음식점의 쌍이 존재한다는것을 알리기 위해 1.0 표시

    # 테스트 데이터 로드
    test_data = []
    with open(root_path + 'negative.csv', 'r') as fd:
        line = fd.readline()
        while line != None and line != '':
            arr = line.split('\t') # test_negative를 \t를 기준 형식에 맞추기
            u = eval(arr[0])[0] # 사용자 n
            test_data.append([u, eval(arr[0])[1]]) #방문한 음식점 추가
            for i in arr[1:]:
                test_data.append([u, int(i)]) #방문하지 않은 음식점 추가
            line = fd.readline()


    return train_data, test_data, user_num, item_num, train_mat

################# DATASET 준비 ##################
train_data, test_data, user_num ,item_num, train_mat = load_all()

train_dataset = BPRData(
		train_data, item_num, train_mat, num_ng, True)
test_dataset = BPRData(
		test_data, item_num, train_mat, 0, False)

train_loader = data.DataLoader(train_dataset,
		batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = data.DataLoader(test_dataset,
		batch_size=test_num_ng+1, shuffle=False, num_workers=0)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


### 모델 생성

In [ ]:
################# 모델 생성 ##################
model = BPR(user_num, item_num, num_factors)
model.cuda()

optimizer = optim.SGD(
            model.parameters(), lr=lr, weight_decay=lamda)

### 훈련

In [ ]:
########################### 훈련 #####################################
count, best_hr = 0, 0
for epoch in range(epochs):
    model.train()
    start_time = time.time()
    train_loader.dataset.ng_sample()

    for user, item_i, item_j in train_loader:
        user = user.cuda()
        item_i = item_i.cuda()
        item_j = item_j.cuda()

        model.zero_grad()
        prediction_i, prediction_j = model(user, item_i, item_j)

        #(prediction_i - prediction_j): user의 item_i 와 item_j의 점수차이 계싼 -> x(uij) = 이 차이가 크고 양수일수록 sigmoid에 넣었을 때 1에 가까워짐
        # sigmoid(): 시그모이드 함수를 적용하여 확률 값으로 변환
        # log -likelihood와 유사한 역할 - 수식을 곱셈에서 덧셈으로 변하게 하여 언더플로우를 방지
        # sum(): 전체 손실을 합산

        # 순전파
        loss = - (prediction_i - prediction_j).sigmoid().log().sum()

        # 역전파
        loss.backward() #손실 함수(loss)의 값을 기준으로 모델의 파라미터에 대한 그래디언트(기울기)를 계산
        # 그래디언트는 모델 파라미터가 현재 값에서 손실을 줄이기 위해 어느 방향으로 얼마나 이동해야 하는지를 결정

        #파라미터 업데이트
        optimizer.step() #앞서 계산된 그래디언트를 사용하여 모델의 파라미터를 업데이트
        #다음 순전파 시 더 나은 예측을 할 수 있도록 도와줌

        # writer.add_scalar('data/loss', loss.item(), count)
        count += 1

    model.eval()
    HR, NDCG = metrics(model, test_loader, top_k)

    elapsed_time = time.time() - start_time
    print("The time elapse of epoch {:03d}".format(epoch) + " is: " +
        time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
    print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

    # if HR > best_hr:
    #   best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
    #   if out:
    #     if not os.path.exists(model_path):
    #       os.mkdir(model_path)
    #     torch.save(model, '{}BPR.pt'.format(model_path))

      # print("End. Best epoch {:03d}: HR = {:.3f}, \
      #   NDCG = {:.3f}".format(best_epoch, best_hr, best_ndcg))


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


The time elapse of epoch 000 is: 00: 00: 10
HR: 0.116	NDCG: 0.050
The time elapse of epoch 001 is: 00: 00: 09
HR: 0.132	NDCG: 0.055
The time elapse of epoch 002 is: 00: 00: 11
HR: 0.140	NDCG: 0.057
The time elapse of epoch 003 is: 00: 00: 12
HR: 0.140	NDCG: 0.058
The time elapse of epoch 004 is: 00: 00: 08
HR: 0.132	NDCG: 0.055
The time elapse of epoch 005 is: 00: 00: 10
HR: 0.132	NDCG: 0.057
The time elapse of epoch 006 is: 00: 00: 09
HR: 0.132	NDCG: 0.057
The time elapse of epoch 007 is: 00: 00: 10
HR: 0.132	NDCG: 0.057
The time elapse of epoch 008 is: 00: 00: 08
HR: 0.140	NDCG: 0.059
The time elapse of epoch 009 is: 00: 00: 10
HR: 0.149	NDCG: 0.061


In [ ]:
torch.save(model.state_dict(), root_path + 'BPR.pt')

## 테스트

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BPR(user_num, item_num, num_factors)
model.to(device)
model.load_state_dict(torch.load(root_path + 'BPR.pt'))
model.eval()

<ipython-input-25-d53ff7c910a3>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(root_path + 'BPR.pt'))


BPR(
  (user_factors): Embedding(12063, 32)
  (item_factors): Embedding(7012, 32)
)

In [26]:
def recommend_restaurants(user_id, model, train_mat, item_num, top_k=10):
    """
    Arg:
    - user_id (int): 추천을 받을 사용자 ID.
    - model (torch.nn.Module): 학습된 BPR 모델.
    - train_mat (scipy.sparse.dok_matrix): 사용자-아이템 희소 행렬, 이미 방문한 음식점 확인에 사용
    - item_num (int): 전체 아이템(음식점) 수.
    - top_k (int): 추천받을 음식점의 수.

    Returns:
    - top_k_items (list of int): 추천된 상위 음식점 ID 리스트.
    """
    # 모든 아이템(음식점)을 대상으로 평가
    item_ids = torch.tensor(range(item_num)).to(device)

    # 사용자 임베딩과 아이템 임베딩을 계산
    user_embedding = model.user_factors(torch.tensor([user_id]).to(device))
    item_embeddings = model.item_factors(item_ids)

    # 내적(matmul)을 통해 각 음식점에 대한 점수 산출 ( [user_embedding * d] [d* item_embeddings] ) ".t()"를 통해 전치행렬로 변환
    # squeeze(0)을 통해 1차원 벡터로 변환 / cpu() - GPU에 있는 텐서를 CPU로 옮김
    scores = torch.matmul(user_embedding, item_embeddings.t()).squeeze(0).cpu().detach().numpy()

    # 이미 방문한 음식점은 추천에서 제외
    print(train_mat[user_id].keys())
    already_visited = list(train_mat[user_id].keys())
    for visited_item in already_visited:
        scores[visited_item[1]] = -np.inf  # 이미 방문한 음식점의 점수를 -무한대로 설정

    # 상위 top_k 음식점을 추출
    top_k_items = np.argsort(scores)[-top_k:][::-1]  # 점수가 높은 순서대로 정렬

    return top_k_items


# 사용자 n에 대해 상위 10개 음식점을 추천
user_id = 3789  # 추천할 사용자 ID
top_k_items = recommend_restaurants(user_id, model, train_mat, item_num, top_k=10)

print(f"Top {len(top_k_items)} 추천 음식점 for 사용자 {user_id}: {top_k_items}")

dict_keys([(0, 50), (0, 64), (0, 75), (0, 94), (0, 211), (0, 292), (0, 325), (0, 333), (0, 361), (0, 484), (0, 503), (0, 614), (0, 683), (0, 756), (0, 781), (0, 952), (0, 975), (0, 1029), (0, 1033), (0, 1076), (0, 1135), (0, 1161), (0, 1212), (0, 1393), (0, 1430), (0, 1483), (0, 1504), (0, 1506), (0, 1841), (0, 1869), (0, 1993), (0, 2000), (0, 2074), (0, 2514), (0, 2923), (0, 3549)])
Top 10 추천 음식점 for 사용자 3789: [2121 1814 6521 6053 1521 2510  114 5269 4371 5349]
